# PENERAPAN TF-IDF DAN COSINE SIMILARITY PADA MESIN PENCARI QUOTES BAHASA INGGRIS BERDASARKAN KATEGORI ATAU AUTHOR

**MASALAH**  
Dalam era digital, terdapat banyak kutipan inspiratif yang tersebar di internet. 
Namun, pengguna sering kesulitan menemukan kutipan tertentu berdasarkan kategori atau penulis yang mereka inginkan. 
Mesin pencari umum biasanya tidak dirancang khusus untuk memberikan hasil pencarian yang relevan terhadap kutipan berdasarkan kategori atau penulis tertentu. 
Hal ini menyebabkan:

* Waktu pencarian yang lama karena hasil yang tidak spesifik.
* Kurangnya kemudahan untuk menemukan kutipan sesuai kebutuhan, seperti motivasi, cinta, atau pendidikan.
* Tidak adanya sistem pencarian yang fokus pada keakuratan dan relevansi kutipan berdasarkan konteks pengguna.


**TUJUAN**  
Tujuan dari project ini adalah:  

* Membuat mesin pencari khusus untuk kutipan (quotes) yang mampu memberikan hasil pencarian yang relevan dan akurat.
* Mengimplementasikan metode TF-IDF dan Cosine Similarity untuk mengukur relevansi antara query pengguna dengan kutipan yang tersedia.
* Memberikan pengalaman pencarian yang efisien bagi pengguna dengan hasil yang lebih terfokus dibandingkan pencarian umum.
* Meningkatkan aksesibilitas kutipan inspirasional yang sesuai dengan kebutuhan atau preferensi pengguna.

**ALUR PENYELESAIAN**

![ALUR PENYELESAIAN](alur-penyelesaian/bagan-alurPenyelesaian.png)


**ANALISIS DATASET DAN FITUR**

In [5]:
# Import library yang diperlukan
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [6]:
# Download data NLTK yang diperlukan
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pramu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pramu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pramu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# Memuat dataset
df = pd.read_csv('quotes.csv')

In [8]:
# Menampilkan informasi dasar tentang dataset
print("Informasi Dataset:")
print(df.info())
print("\nContoh Data:")
print(df.head())

Informasi Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499709 entries, 0 to 499708
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   quote     499708 non-null  object
 1   author    497956 non-null  object
 2   category  499646 non-null  object
dtypes: object(3)
memory usage: 11.4+ MB
None

Contoh Data:
                                               quote  \
0  I'm selfish, impatient and a little insecure. ...   
1  You've gotta dance like there's nobody watchin...   
2  You know you're in love when you can't fall as...   
3  A friend is someone who knows all about you an...   
4  Darkness cannot drive out darkness: only light...   

                                              author  \
0                                     Marilyn Monroe   
1                                  William W. Purkey   
2                                          Dr. Seuss   
3                                     Elbert Hubbard   

In [9]:
# EDA Dasar
print("\nJumlah penulis unik:", df['author'].nunique())
print("Jumlah kategori unik:", df['category'].nunique())
print("\nDistribusi kategori:")
print(df['category'].value_counts())


Jumlah penulis unik: 117238
Jumlah kategori unik: 367915

Distribusi kategori:
category
education, happiness, hope, inspirational, intelligence, knowledge, life, love, philosophy, quotes, truth, wisdom    1648
humorous                                                                                                             1063
happiness                                                                                                            1013
friendship                                                                                                            901
prayer                                                                                                                873
                                                                                                                     ... 
amsterdam, words                                                                                                        1
kvothe, love, mother-love, vashet, words                 

**PEMROSESAN TEKS DAN IMPLEMENTASI MODEL**

In [11]:
# Fungsi preprocessing teks
def preprocess_text(text):
    """
    Melakukan preprocessing pada teks input dengan tahapan:
    1. Melakukan preprocessing pada teks input dengan penanganan nilai NaN
    2. Konversi ke huruf kecil
    3. Menghapus karakter khusus dan angka
    3. Tokenisasi
    5. Menghapus stopwords
    6. Lemmatisasi
    """
    # Penanganan nilai NaN atau non-string
    if pd.isna(text) or not isinstance(text, str):
        return ""
    
    # Konversi ke huruf kecil
    text = text.lower()
    
    # Menghapus karakter khusus dan angka
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Tokenisasi
    tokens = word_tokenize(text)
    
    # Menghapus stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatisasi
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(tokens)

In [12]:
# Membersihkan dataset dari nilai NaN
print("Jumlah nilai NaN dalam kolom 'quote':", df['quote'].isna().sum())
print("Jumlah total data sebelum pembersihan:", len(df))

Jumlah nilai NaN dalam kolom 'quote': 1
Jumlah total data sebelum pembersihan: 499709


In [13]:
# Menghapus baris dengan nilai NaN
df = df.dropna(subset=['quote'])
print("Jumlah total data setelah pembersihan:", len(df))

Jumlah total data setelah pembersihan: 499708


In [14]:
# Menerapkan preprocessing pada quotes
df['processed_quote'] = df['quote'].apply(preprocess_text)

In [15]:
# Membuat vektor TF-IDF untuk quotes
tfidf = TfidfVectorizer()
quote_vectors = tfidf.fit_transform(df['processed_quote'])

In [16]:
authors = df['author'].str.lower().unique()
categories = df['category'].str.lower().unique()

In [24]:
def search_quotes(query, search_type='content', top_n=5):
    """
    Mencari quotes berdasarkan konten, penulis, atau kategori

    Parameter:
    query (str): Kata kunci pencarian
    search_type (str): Tipe pencarian ('quote', 'author', 'category')
    top_n (int): Jumlah hasil yang akan dikembalikan

    Returns:
    list: Quotes yang paling cocok beserta detailnya
    """
    if search_type == 'content':
        # Preprocessing query
        processed_query = preprocess_text(query)

        # Transformasi query ke vektor TF-IDF
        query_vector = tfidf.transform([processed_query])

        # Menghitung cosine similarity
        similarities = cosine_similarity(query_vector, quote_vectors).flatten()

        # Mendapatkan N hasil teratas
        top_indices = similarities.argsort()[-top_n:][::-1]

        results = []
        for idx in top_indices:
            results.append({
                'quote': df.iloc[idx]['quote'],
                'author': df.iloc[idx]['author'],
                'category': df.iloc[idx]['category'],
                'similarity': similarities[idx]
            })

        return results
    elif search_type in ['author', 'category']:
        # Filter berdasarkan penulis atau kategori
        matches = df[df[search_type].fillna('').str.lower().str.contains(query.lower())]
        return matches.head(top_n)[['quote', 'author', 'category']].to_dict('records')
    else:
        # Pencarian berdasarkan konten dengan kecocokan parsial
        matches = df[df['processed_quote'].str.contains(query.lower(), na=False)]
        return matches.head(top_n)[['quote', 'author', 'category']].to_dict('records')

**PERFORMA MODEL**

In [50]:
# Fungsi untuk mengukur performa pencarian dengan penanganan NA/NaN
def test_search_performance():
    """
    Menguji performa pencarian dengan berbagai tipe query
    dan menangani kemungkinan nilai NA/NaN
    """
    test_queries = [
        ('love', 'quote'),
        ('Max Lucado', 'author'),
        ('motivation', 'category')
    ]
    
    print("=== Pengujian Performa Pencarian ===")
    
    for query, search_type in test_queries:
        try:
            print(f"\nMengetes pencarian untuk:")
            print(f"Query: '{query}'")
            print(f"Tipe pencarian: {search_type}")
            
            # Mengukur waktu eksekusi
            start_time = time.time()
            
            if search_type == 'content':
                # Pencarian berdasarkan konten quote
                processed_query = preprocess_text(query)
                query_vector = tfidf.transform([processed_query])
                similarities = cosine_similarity(query_vector, quote_vectors).flatten()
                top_indices = similarities.argsort()[-5:][::-1]
                results = []
                for idx in top_indices:
                    results.append({
                        'quote': df.iloc[idx]['quote'],
                        'author': df.iloc[idx]['author'],
                        'category': df.iloc[idx]['category'],
                        'similarity': similarities[idx]
                    })
            else:
                # Pencarian berdasarkan author atau category
                # Menggunakan metode yang aman untuk handling NA/NaN
                mask = df[search_type].fillna('').str.lower().str.contains(query.lower())
                matches = df[mask].head(5)
                results = matches[['quote', 'author', 'category']].to_dict('records')
            
            execution_time = time.time() - start_time
            
            print(f"Waktu eksekusi: {execution_time:.4f} detik")
            print(f"Jumlah hasil: {len(results)}")
            
            # Menampilkan contoh hasil pertama
            if results:
                print("\nContoh hasil pertama:")
                print(f"Quote: {results[0]['quote']}")
                print(f"Author: {results[0]['author']}")
                print(f"Category: {results[0]['category']}")
            else:
                print("Tidak ditemukan hasil")
                
        except Exception as e:
            print(f"Error saat menguji query '{query}' ({search_type}): {str(e)}")
            continue

In [52]:
# Menjalankan pengujian
print("Informasi Dataset:")
print(f"Total quotes: {len(df)}")
print(f"Jumlah quotes valid (non-NA): {df['quote'].notna().sum()}")
print(f"Jumlah author unik: {df['author'].nunique()}")
print(f"Jumlah kategori unik: {df['category'].nunique()}")
print("\nMemulai pengujian performa...")
test_search_performance()

Informasi Dataset:
Total quotes: 499708
Jumlah quotes valid (non-NA): 499708
Jumlah author unik: 117238
Jumlah kategori unik: 367915

Memulai pengujian performa...
=== Pengujian Performa Pencarian ===

Mengetes pencarian untuk:
Query: 'love'
Tipe pencarian: quote
Error saat menguji query 'love' (quote): name 'time' is not defined

Mengetes pencarian untuk:
Query: 'Max Lucado'
Tipe pencarian: author
Error saat menguji query 'Max Lucado' (author): name 'time' is not defined

Mengetes pencarian untuk:
Query: 'motivation'
Tipe pencarian: category
Error saat menguji query 'motivation' (category): name 'time' is not defined


In [37]:
from IPython.display import HTML, display

def create_search_box():
    """
    Membuat tampilan search box sederhana untuk mencari quotes
    """
    
def smart_search(query):
    """
    Fungsi pencarian pintar yang bisa mendeteksi tipe pencarian dari input
    """
    query = query.strip().lower()
    
    # Cek apakah query adalah penulis atau kategori
    if query in authors:
        search_type = 'author'
        search_query = query
    elif query in categories:
        search_type = 'category'
        search_query = query
    else:
        search_type = 'content'
        search_query = query
    
    if not search_query:
        print("Kata kunci pencarian tidak boleh kosong!")
        return
    
    results = search_quotes(search_query, search_type)
    
    # Menampilkan hasil
    print(f"\nHasil pencarian untuk '{search_query}' (Tipe: {search_type}):")
    if results:
        for i, result in enumerate(results, 1):
            print(f"\n{i}. Quote: {result['quote']}")
            print(f"   Author: {result['author']}")
            print(f"   Category: {result['category']}")
            if 'similarity' in result:
                print(f"   Similarity: {result['similarity']:.4f}")
    else:
        print("Tidak ditemukan hasil yang cocok.")

In [39]:
# Fungsi untuk memulai pencarian
def start_search():
    create_search_box()
    while True:
        print("\n" + "="*50)
        query = input("Masukkan kata kunci pencarian (ketik 'exit' untuk keluar): ")
        
        if query.lower() == 'exit':
            print("Terima kasih telah menggunakan search engine quotes!")
            break
            
        smart_search(query)

In [41]:
# Jalankan pencarian
start_search()

Masukkan kata kunci pencarian (ketik 'exit' untuk keluar):  Romance



Hasil pencarian untuk 'romance' (Tipe: category):

1. Quote: When someone loves you, the way they talk about you is different. You feel safe and comfortable.
   Author: Jess C. Scott, The Intern
   Category: desire, emotion, friendship, honesty, imagination, individuality, life, love, passion, reality, relationships, romance, truth, wisdom, wise-words

2. Quote: I no longer believed in the idea of soul mates, or love at first sight. But I was beginning to believe that a very few times in your life, if you were lucky, you might meet someone who was exactly right for you. Not because he was perfect, or because you were, but because your combined flaws were arranged in a way that allowed two separate beings to hinge together.
   Author: Lisa Kleypas, Blue-Eyed Devil
   Category: love, relationships, romance

3. Quote: The best love is the kind that awakens the soul and makes us reach for more, that plants a fire in our hearts and brings peace to our minds. And that's what you've given me

Masukkan kata kunci pencarian (ketik 'exit' untuk keluar):  Lady gaga



Hasil pencarian untuk 'lady gaga' (Tipe: author):

1. Quote: Some women choose to follow men, and some women choose to follow their dreams. If you're wondering which way to go, remember that your career will never wake up and tell you that it doesn't love you anymore.
   Author: Lady Gaga
   Category: love, romance, work

2. Quote: Love is like a brick. You can build a house, or you can sink a dead body.
   Author: Lady Gaga
   Category: brick, dead, house, humor, lady-gaga, love

3. Quote: Well my music was different in high school; I was singing about love—you know, things I don't care about anymore.
   Author: Lady Gaga
   Category: high-school, lady-gaga, love, music, school

4. Quote: It doesn't matter who you are, or where you come from, or how much money you've got in your pocket. You have your own destiny and your own life ahead of you.
   Author: Lady Gaga
   Category: inspirational, life

5. Quote: If you dont have any shadows you're not in the light
   Author: Lady Gaga
   

Masukkan kata kunci pencarian (ketik 'exit' untuk keluar):  never surrender



Hasil pencarian untuk 'never surrender' (Tipe: content):

1. Quote: Never Give up, never surrender.
   Author: Nancy J. Cohen
   Category: mystery, paranormal-romance, romance
   Similarity: 0.8741

2. Quote: Surrender your expectations. Surrender your doubts. Surrender your fears. Surrender your strengths. Surrender your anger. Surrender your control.
   Author: Yasmine Galenorn, Darkling
   Category: darkling, paranormal, paranormal-romance, vampire, yasmine-galenorn
   Similarity: 0.8503

3. Quote: Surrender to God.
   Author: Lailah Gifty Akita
   Category: christain, god, surrender
   Similarity: 0.7914

4. Quote: You have to surrender yourself to the experience.
   Author: Thomm Quackenbush, Artificial Gods
   Category: experience, surrender
   Similarity: 0.7419

5. Quote: Just surrender to the story.
   Author: A.D. Posey
   Category: ad-posey, inspirational, story, storytellers, storytelling, surrender, write, writers, writing, writing-quotes
   Similarity: 0.7392



Masukkan kata kunci pencarian (ketik 'exit' untuk keluar):  exit


Terima kasih telah menggunakan search engine quotes!


**DISKUSI HASIL**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Mesin pencari yang diimplementasikan berhasil menangani berbagai jenis pencarian dengan baik, mencakup pencarian berbasis konten, penulis, dan kategori. Penggunaan algoritma TF-IDF dan cosine similarity terbukti efektif dalam menentukan relevansi untuk pencarian berbasis konten, memberikan hasil yang sesuai dengan query pengguna. Untuk pencarian berdasarkan penulis dan kategori, pendekatan pencocokan string langsung digunakan, yang memberikan presisi lebih tinggi karena langsung membandingkan query dengan data yang ada tanpa transformasi lebih lanjut. Tahap preprocessing data, yang melibatkan konversi ke huruf kecil, penghapusan karakter khusus, stopword, serta proses lemmatisasi, berkontribusi signifikan dalam meningkatkan kualitas hasil pencarian dengan memastikan data lebih bersih dan seragam. Sistem ini dirancang untuk menangani berbagai jenis query secara efisien, termasuk mengelola nilai kosong (NaN) pada dataset, sehingga menghasilkan pengalaman pencarian yang lebih andal.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Selain itu, implementasi evaluasi menunjukkan bahwa mesin pencari memiliki performa yang baik dalam mengolah query dengan waktu eksekusi yang cepat. Hasil yang relevan disertai dengan atribut tambahan, seperti kemiripan untuk pencarian berbasis konten, memberikan informasi yang lebih lengkap kepada pengguna. Secara keseluruhan, sistem ini telah memenuhi tujuan awal, yaitu menjadi mesin pencari yang fleksibel, presisi, dan efisien untuk dataset kutipan.

**KESIMPULAN**  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Dari hasil implementasi dan pengujian, mesin pencari ini terbukti efektif dalam menangani berbagai kriteria pencarian, baik berbasis konten, penulis, maupun kategori. Algoritma TF-IDF yang digabungkan dengan cosine similarity memberikan hasil pencarian yang relevan untuk konten, sedangkan pencocokan string langsung memberikan akurasi tinggi untuk pencarian berdasarkan penulis dan kategori. Tahap preprocessing data memainkan peran penting dalam meningkatkan kualitas data, sehingga mendukung proses pencarian yang lebih baik. Sistem ini dirancang agar dapat diskalakan dan mampu menangani ukuran dataset saat ini secara efektif. Dengan fitur yang ada, mesin pencari ini menjadi solusi yang andal untuk pencarian berbasis kutipan, memberikan hasil yang relevan, cepat, dan sesuai dengan kebutuhan pengguna.